# Applied Data Analysis Project
**Team**: ToeStewBrr - Alexander Sternfeld, Marguerite Thery, Antoine Bonnet, Hugo Bordereaux

**Dataset**: CMU Movie Summary Corpus


## CoreNLP Analysis

[**CoreNLP**](https://nlp.stanford.edu/software/) is an incredible natural language processing toolkit created at Stanford University. CoreNLP is applied through a **pipeline** of sequential analysis steps called annotators. The full list of available annotators is available [here](https://stanfordnlp.github.io/CoreNLP/annotators.html). 

As described by its creators: 

*"CoreNLP is your one stop shop for natural language processing in Java! CoreNLP enables users to derive linguistic annotations for text, including token and sentence boundaries, parts of speech, named entities, numeric and time values, dependency and constituency parses, coreference, sentiment, quote attributions, and relations. CoreNLP currently supports 8 languages: Arabic, Chinese, English, French, German, Hungarian, Italian, and Spanish."* 

You can create your own pipeline to extract the desired information. You can try it out for yourself in this [online shell](https://corenlp.run).

### Loading data
We first load data files and download the pre-processed dataframes. 

In [1]:
from zipfile import ZipFile
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np

from load_data import *
from coreNLP_analysis import *
from extraction import *

download_data(coreNLP=False)
plot_df = load_plot_df()
movie_df = load_movie_df()
char_df = load_char_df()
names_df = load_names_df()
cluster_df = load_cluster_df()

### 1. Exploring pre-processed CoreNLP data

The authors of the Movie CMU dataset used CoreNLP to parse each plot summary to extract various linguistic insights. In this section, we explore how much information we can gather from these pre-processed files. 

We will use *Harry Potter*'s character throughout this section.

#### 1.1. Character data

For any character, we first extract related information from the provided name clusters and character metadata.

In [2]:
# Given character, extract all pre-processed dataframe data
char_name = 'Harry Potter'
movie_ids = list(char_df[char_df['Character name'] == char_name]['Wikipedia ID'])

print('Movies with character', char_name, ':')
print('\tMovie IDs:', movie_ids)

movie_id = movie_ids[3]
movie_name = movie_df.loc[movie_df['Wikipedia ID'] == movie_id]['Name'].iloc[0]

print('Selecting as example: \n\tMovie ID:', movie_id, '\n\tMovie title:', movie_name)


Movies with character Harry Potter :
	Movie IDs: [858575, 667372, 670407, 31941988, 9834441, 667368, 667371, 667361, 667361]
Selecting as example: 
	Movie ID: 31941988 
	Movie title: Harry Potter and the Deathly Hallows – Part 2


#### 1.2. Extracting sentences

We now extract information from the CoreNLP plot summary analysis. The authors of the dataset stored the analysis output of each movie into a `.xml` file. Each file has a tree structure detailing each word of each sentence as well as the parsed sentence in tree form. 

We now extract all parsed sentences from the `.xml` files. 

A **parsed sentence** is a syntactic analysis tree, where each word is a leaf tagged by its lexical function (e.g. *VBZ* for verbs or *DT* for determinants). Semantic interactions between different words are also indicated within the structure of the tree. 

In [3]:
# Extract the tree of xml file and all parsed sentences
tree = get_tree(movie_id)
sentences = get_parsed_sentences(tree)

# Picking the fifth sentence as example
parsed_str = sentences[5]
print(parsed_str)
print_tree(parsed_str)

(ROOT (S (PP (IN In) (NP (NP (NNP Bellatrix) (POS 's)) (NN vault))) (, ,) (NP (NNP Harry)) (VP (VBZ discovers) (SBAR (S (NP (DT the) (NNP Horcrux)) (VP (VBZ is) (NP (NP (NNP Helga) (NNP Hufflepuff) (POS 's)) (NN cup)))))) (. .))) 
                                                ROOT                                                 
                                                 |                                                    
                                                 S                                                   
                _________________________________|_________________________________________________   
               |             |    |                               VP                               | 
               |             |    |        _______________________|____                            |  
               |             |    |       |                           SBAR                         | 
               |             |    |       |         

#### 1.3. Extracting characters

We also want to extract all character names directly from the xml file. Note that we aggregate consecutive words tagged as NNP (noun, proper, singular) as the same character name (this assumes that plot summaries never contain two distinct names side by side without delimiting punctuation). This is a reasonable assumption since list of names are almost always separated by commas. 

In [4]:
characters = get_characters(tree)
print(characters[:20])

['Voldemort', 'Albus Dumbledore', 'Severus Snape', 'Dobby', 'Harry Potter', 'Ron', 'Hermione', 'Griphook', 'Harry', 'Ollivander', 'Ollivander', 'Draco Malfoy', 'Malfoy', 'Harry', 'Harry', 'Helga Hufflepuff', 'Griphook', 'Harry', 'Voldemort', 'Griphook']


Notice that some characters are sometimes mentioned by their full name, and sometimes by a partial name (e.g. Harry Potter is most often mentioned as simply Harry). To get a more precise idea of how many times each character is mentioned, we wish to denote each character by their full name, i.e. the longest version of their name that appears in the plot summary. 

*NOTE*: The dataset has the character metadata of only a third of the movies, so we need to extract full names from the plot summary itself and not the provided dataframes. 

To optimize full name lookup, for each plot summary we construct a dictionary which stores as key every partial name mentioned, and as corresponding values the full name of each character.  

In [5]:
char_name = 'Albus'
full_name = get_full_name(char_name, characters)
print('Example: the full name of "{}" is "{}".'.format(char_name,full_name))
print('Full name dictionary:', full_name_dict(characters))

Example: the full name of "Albus" is "Albus Dumbledore".
Full name dictionary: {'Voldemort': 'Voldemort', 'Albus Dumbledore': 'Albus Dumbledore', 'Severus Snape': 'Severus Snape', 'Dobby': 'Dobby', 'Harry Potter': 'Harry Potter', 'Ron': 'Ron', 'Hermione': 'Hermione Weasley', 'Griphook': 'Griphook', 'Harry': 'Harry Potter', 'Ollivander': 'Ollivander', 'Draco Malfoy': 'Draco Malfoy', 'Malfoy': 'Draco Malfoy', 'Helga Hufflepuff': 'Helga Hufflepuff', 'Rowena Ravenclaw': 'Rowena Ravenclaw', 'Hogsmeade': 'Hogsmeade', 'Aberforth Dumbledore': 'Aberforth Dumbledore', 'Ariana': 'Ariana', 'Neville Longbottom': 'Neville Longbottom', 'Snape': 'Severus Snape', 'Minerva McGonagall': 'Minerva McGonagall', 'Luna Lovegood': 'Luna Lovegood', 'Helena Ravenclaw': 'Helena Ravenclaw', 'Gregory Goyle': 'Gregory Goyle', 'Blaise Zabini': 'Blaise Zabini', 'Nagini': 'Nagini', 'Fred': 'Fred', 'Lily': 'Lily', 'James': 'James', 'Dumbledore': 'Albus Dumbledore', 'Neville': 'Neville Longbottom', 'Molly Weasley': 'Moll

We can now extract the most mentioned characters in any plot summary, in descending order of frequency. We can then see that Harry Potter is indeed the main character of the movie, as he is mentioned 26 times, more than any other character in the summary.  

In [6]:
char_mentions = most_mentioned(movie_id)
print(char_mentions)

[('Harry Potter', 26), ('Voldemort', 21), ('Severus Snape', 11), ('Ron', 6), ('Hermione Weasley', 6), ('Albus Dumbledore', 5), ('Griphook', 3), ('Draco Malfoy', 3), ('Neville Longbottom', 3), ('Nagini', 3), ('Ollivander', 2), ('Lily', 2), ('Dobby', 1), ('Helga Hufflepuff', 1), ('Rowena Ravenclaw', 1), ('Hogsmeade', 1), ('Aberforth Dumbledore', 1), ('Ariana', 1), ('Minerva McGonagall', 1), ('Luna Lovegood', 1), ('Helena Ravenclaw', 1), ('Gregory Goyle', 1), ('Blaise Zabini', 1), ('Fred', 1), ('James', 1), ('Molly Weasley', 1), ('Ginny Potter', 1)]


 #### 1.4. Extracting interactions

We are also interested in character interactions. We can use the number of common mentions of two characters in the same sentence as a proxy for the number of interactions. For any movie, we find the number of common mentions (i.e. interactions) for each pair of characters. 

In [7]:
char_pairs = character_pairs(movie_id, plot_df)
print(char_pairs[:10])

[(('Hermione Weasley', 'Ron'), 4), (('Harry Potter', 'Voldemort'), 4), (('Albus Dumbledore', 'Voldemort'), 3), (('Albus Dumbledore', 'Severus Snape'), 2), (('Harry Potter', 'Hermione Weasley'), 2), (('Harry Potter', 'Ron'), 2), (('Nagini', 'Voldemort'), 2), (('Harry Potter', 'Lily'), 2), (('Albus Dumbledore', 'Harry Potter'), 2), (('Severus Snape', 'Voldemort'), 1)]


In [8]:
main_interaction = character_pairs(movie_id, plot_df)[0][0]
print('Main interaction in the movie:', main_interaction)

Main interaction in the movie: ('Hermione Weasley', 'Ron')


#### 1.5. Extracting characters and interactions of all movies

We will now use the above code to obtain the main character and main interaction for every plot summary. 

*NOTE*: This code takes a while to run, so you can load the analysis from a pre-processed file instead.  

In [9]:
# NOTE: If we've already run this code, we can load the dataframe from a file
plot_char_filename = 'Data/MovieSummaries/plot_characters.csv'
pairs_df = pd.read_csv(plot_char_filename, sep='\t', index_col=0)
pairs_df

,Wikipedia ID,Summary,Main character,Number of mentions,Main interaction,Number of interactions
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...",Shlykov,1.0,"('Lyosha', 'Shlykov')",1.0
1,31186339,The nation of Panem consists of a wealthy Capi...,Katniss,18.0,"('Katniss', 'Peeta Mellark')",2.0
2,20663735,Poovalli Induchoodan is sentenced for six yea...,Maranchery Karunakara Menon,9.0,"('Manapally Madhavan Nambiar', 'judge Menon')",1.0
3,2231378,"The Lemon Drop Kid , a New York City swindler,...",Charley,18.0,NaN,NaN
4,595909,Seventh-day Adventist Church pastor Michael Ch...,Lindy,7.0,"('Azaria', 'Lindy')",1.0
...,...,...,...,...,...,...
42298,34808485,"The story is about Reema , a young Muslim scho...",Reema,1.0,"('Muslim', 'Reema')",1.0
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look...",Leo Andreyev,7.0,NaN,NaN
42300,35102018,American Luthier focuses on Randy Parsons’ tra...,Randy Parsons,4.0,NaN,NaN
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se...",Abdur Rehman Khan,9.0,"('Abdur Rehman Khan', 'Amina')",1.0


In [10]:
# Otherwise: get main character and number of mentions for each movie and store it into a file (takes a while to run)
if not os.path.exists(plot_char_filename):
    pairs_df = plot_df.copy(deep=True)
    pairs_df['Main character'] = pairs_df['Wikipedia ID'].apply(most_mentioned)
    pairs_df['Number of mentions'] = pairs_df['Main character'].apply(lambda x: np.nan if x is None else x[0][1])
    pairs_df['Main character'] = pairs_df['Main character'].apply(lambda x: np.nan if x is None else x[0][0])

    # Get main pairs of characters for each movie and number of interactions 
    pairs_df['Main interaction'] = pairs_df['Wikipedia ID'].apply(lambda x: character_pairs(x, plot_df))
    pairs_df['Number of interactions'] = pairs_df['Main interaction'].apply(lambda x: np.nan if x is None else x[0][1])
    pairs_df['Main interaction'] = pairs_df['Main interaction'].apply(lambda x: np.nan if x is None else x[0][0])

    # Store data into csv file
    pairs_df.to_csv(plot_char_filename, sep='\t')
    pairs_df

In conclusion, the coreNLP files provided with the datasets are useful to extract the characters mentioned. 

 However, our goal is to extract love relationships as well as the persona of characters in love. Using common mentions as a proxy for love relationships is a vulgar approximation and so we must run our own NLP analysis on the plot summaries to extract useful information. 

### 2. Custom CoreNLP Analysis

We now use a **custom CoreNLP pipeline** to analyze the plot summaries. For now, due to the weakness of our available computing power, we only analyze romantic comedy movies. 


#### 2.1. Data preparation

We extract the romantic comedy plot summaries that we will pass through our pipeline and store them as `.txt` files to be able to run them through the new coreNLP pipeline. 

In [11]:
# For later use: romance_genres = ['Romantic comedy', 'Romance Film', 'Romantic drama', 'Romantic fantasy', 'Romantic thriller']

# Get a dataframe with romantic movies and their corresponding plots
romance_genres = ['Romantic comedy'] 
rom_com_plots = get_plots(romance_genres, movie_df, plot_df)
#display(rom_com_plots)

# Store each plot summary as .txt file
for index, row in rom_com_plots.iterrows():
    # If directory doesn't exist, create it
    if not os.path.exists('Data/MovieSummaries/RomancePlots'):
        os.makedirs('Data/MovieSummaries/RomancePlots')
    with open("Data/MovieSummaries/RomancePlots/{}.txt".format(row['Wikipedia ID']), 'w', encoding='utf8') as f:
        if type(row['Summary']) == str:
            f.write(row['Summary'])
            f.close()

#### 2.1. Custom CoreNLP pipeline

Our custom pipeline consists of the following annotators: 

1. [Tokenization (tokenize)](https://stanfordnlp.github.io/CoreNLP/tokenize.html): Turns the whole text into tokens. 

2. [Parts Of Speech (POS)](https://stanfordnlp.github.io/CoreNLP/pos.html): Tags each token with part of speech labels (e.g. determinants, verbs and nouns). 

3. [Lemmatization (lemma)](https://stanfordnlp.github.io/CoreNLP/lemma.html): Reduces each word to its lemma (e.g. *was* becomes *be*). 

4. [Named Entity Recognition (NER)](https://stanfordnlp.github.io/CoreNLP/ner.html): Identifies named entities from the text, including characters, locations and organizations. 

5. [Constituency parsing (parse)](https://stanfordnlp.github.io/CoreNLP/parse.html): Performs a syntactic analysis of each sentence in the form of a tree. 

6. [Coreference resolution (coref)](https://stanfordnlp.github.io/CoreNLP/coref.html): Aggregates mentions of the same entities in a text (e.g. when 'Harry' and 'he' refer to the same person). 

7. [Dependency parsing (depparse)](https://stanfordnlp.github.io/CoreNLP/depparse.html): Syntactic dependency parser. 

8. [Natural Logic (natlog)](https://stanfordnlp.github.io/CoreNLP/natlog.html): Identifies quantifier scope and token polarity. Required as preliminary for OpenIE. 

9. [Open Information Extraction (OpenIE)](https://stanfordnlp.github.io/CoreNLP/openie.html): Identifies relation between words as triples *(subject, relation, object of relation)*. We use this to extract relationships between characters, as well as character traits. 

10. [Knowledge Base Population (KBP)](https://stanfordnlp.github.io/CoreNLP/kbp.html): Identifies meaningful relation triples. 


#### 2.2. Running our pipeline

We now run our own CoreNLP analysis on the plot summaries. This allows us to extract love relationships from the plot summaries much more accurately.

**Goal**: Run our custom CoreNLP pipeline. 

**Recommendation**: Be careful about memory storage (takes a lot of memory to run!)

**Prerequisite**: [java](https://www.java.com). 

**Installation steps**:
1. Download the CoreNLP toolkit [here](https://stanfordnlp.github.io/CoreNLP/download.html).

2. Data preparation: Extract plot summaries for romantic comedies into `.txt` files. Create a filelist containing the name of all the files which need to be processed using the following command: 

        find RomancePlots/*.txt > filelist.txt

3. Change directory (`cd`) into the downloaded `stanford-corenlp` directory. 
        
4. Run the custom CoreNLP pipeline via your terminal using the following command:

        java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,pos,lemma,ner,parse,coref,depparse,natlog,openie,kbp -coref.md.type RULE -filelist filelist.txt -outputDirectory RomancePlotsOutputs/ -outputFormat xml

The analysis outputs are now stored as `.xml` files in the `RomancePlotsOutputs` directory. We now unzip them. RomancePlotsOutputs has 1491 readable files. 

In [12]:
# Extract all the romance plots xml files
# If r`RomancePlotsOutputs directory doesn't exist, create it
if not os.path.exists('Data/CoreNLP/RomancePlotsOutputs'):
   with ZipFile('CoreNLP/RomanceOutputs.zip', 'r') as zipObj:
      zipObj.extractall('')

### 3. Extracting data

Now that we have run the coreNLP pipeline and that the analysis of each movie has been a stored into a .xml output file, we can extract the information from these files. 

We will first extract the attributes and actions related to entities in the plot summaries. We will extract verbs and attributes independently. 
Agent verb: character does the action
Patient verb: character is the object of the action
Attributes: character attributes

**Dependency parsing extraction**
| Relation | Description |  Type  |  Example |
|---|---|---|---|
| obl:agent | Agent | Agent verb | 'They were rescued by Dumbledore' -> obl:agent(rescued, Dumbledore) |
| nsubj  | Nominal subject | Agent verb | 'Harry confronts Snape' -> nsubj(confronts, Harry) |
| nsubj:pass | Passive nominal subject | Patient verb | 'Goyle casts a curse and is burned to death' -> nsubj:pass(burned, Goyle)|
| nsubj:xsubj | Indirect nominal subject | Patient verb | 'Goyle casts a curse and is unable to control it' -> nsubj:xsubj(control, Goyle)|
| obj |  Direct object | Patient verb | 'To protect Harry' -> obj(protect, Harry) |
| appos | Appositional modifier | Attribute | 'Harry's mother, Lily' -> appos(mother, Lily) |
| amod | Adjectival modifier | Attribute | 'After burrying Dobby' -> amod(Dobby, burrying) |
| nmod:poss | Possessive nominal modifier | Attribute | 'Snape's memories' -> nmod:poss(memories, Snape) |
| nmod:of | 'Of' nominal modifier | Attribute |'With the help of Griphook' -> nmod:of(help, Griphook) |

We will also extract KBP outputs, which stores data including the main role, spouse, age and religion for each character if specified. 

**KBP Extraction**
| Attributes | Relation name | 
|---|---|
| Main role | per:title |
| Marital relationship | per:spouse  |  
| Age  | per:age | 
| Religion  | per:religion | 
| Death | per:cause_of_death |

The [KBP documentation](https://stanfordnlp.github.io/CoreNLP/kbp.html) contains a description of all available KBP tags.

**Running extraction**

tree = ET.parse(example_filename)
descriptions_df, relations_df = get_descriptions_relations(tree)
display(descriptions_df)
display(relations_df)

In [13]:
description_path = 'Data/CoreNLP/descriptions.csv'
relations_path = 'Data/CoreNLP/relations.csv'

if not os.path.exists(description_path) and not os.path.exists(relations_path):

    # Extract descriptions and relations from all xml files
    output_dir = 'Data/CoreNLP/PlotsOutputs'
    descriptions, relations = extract_descriptions_relations(output_dir)

    # Save descriptions and relations into csv files
    descriptions.to_csv(description_path, sep='\t')
    relations.to_csv(relations_path, sep='\t')

# If we've already run the extraction, we can load the dataframe from a file
else:
    non_rom_descriptions = pd.read_csv(
        description_path, sep='\t', index_col=0, low_memory=False)
    non_rom_relations = pd.read_csv(
        relations_path, sep='\t', index_col=0, low_memory=False)


Same thing for romantic movies

In [14]:
romance_description_path = 'Data/CoreNLP/romance_descriptions.csv'
romance_relations_path = 'Data/CoreNLP/romance_relations.csv'

if not os.path.exists(romance_description_path) and not os.path.exists(romance_relations_path):

    # Extract descriptions and relations from all romance xml files
    romance_output_dir = 'Data/CoreNLP/RomancePlotsOutputs'

    # Remove file '43849.xml' from the directory, as it is not a valid xml file
    if os.path.exists(f'{romance_output_dir}/43849.xml'):
        os.remove(f'{romance_output_dir}/43849.xml')

    romance_descriptions, romance_relations = extract_descriptions_relations(
        romance_output_dir)

    # Save descriptions and relations into csv files
    romance_descriptions.to_csv(romance_description_path, sep='\t')
    romance_relations.to_csv(romance_relations_path, sep='\t')

# If we've already run the extraction, we can load the dataframe from a file
else:
    rom_descriptions = pd.read_csv(
        romance_description_path, sep='\t', index_col=0, low_memory=False)
    rom_relations = pd.read_csv(
        romance_relations_path, sep='\t', index_col=0, low_memory=False)


We now merge the romance and non-romance descriptions and relations in a single DataFrame. 

In [15]:
# Merge the descriptions of romance and non-romance movies
non_rom_descriptions['romance'] = False
rom_descriptions['romance'] = True
descriptions = pd.concat(
    [non_rom_descriptions, rom_descriptions], ignore_index=True)

# Merge the relations of romance and non-romance movies
non_rom_relations['romance'] = False
rom_relations['romance'] = True
relations = pd.concat(
    [non_rom_relations, rom_relations], ignore_index=True)



### 4. Processing extracted data


#### 4.1. Processing descriptions

We now pre-process the extracted character analysis, merge it with the pre-existing character and movie metadata and store it into a cute data file. 

1. Convert to lists and aggregate all descriptions. 

In [16]:
descriptions['attributes'] = descriptions['attributes'].apply(
    lambda x: x if pd.isnull(x) else x[1:-1].replace("'", "").replace(" ", "").split(','))
descriptions['agent_verbs'] = descriptions['agent_verbs'].apply(
    lambda x: x if pd.isnull(x) else x[1:-1].replace("'", "").replace(" ", "").split(','))
descriptions['patient_verbs'] = descriptions['patient_verbs'].apply(
    lambda x: x if pd.isnull(x) else x[1:-1].replace("'", "").replace(" ", "").split(','))

descriptions['all_descriptions'] = descriptions[['agent_verbs', 'patient_verbs', 'attributes', 'title']].apply(
    lambda x: [item for sublist in x if type(sublist) == list for item in sublist], axis=1)


2. Lemmatize all words to their lexical root and verbs to their infinitive present tense. 

In [17]:
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

def lemmatize(x): return [lem.lemmatize(
    lem.lemmatize(word), 'v') for word in x]

descriptions['agent_verbs'] = descriptions['agent_verbs'].apply(
    lambda x: lemmatize(x) if type(x) == list else x)
descriptions['patient_verbs'] = descriptions['patient_verbs'].apply(
    lambda x: lemmatize(x) if type(x) == list else x)
descriptions['attributes'] = descriptions['attributes'].apply(
    lambda x: lemmatize(x) if type(x) == list else x)
descriptions['all_descriptions'] = descriptions['all_descriptions'].apply(
    lambda x: lemmatize(x) if type(x) == list else x)


3. Synchronize character names to the character metadata. 

In [19]:
descriptions

,movie_id,character,agent_verbs,patient_verbs,attributes,title,religion,age,death,children,romance,all_descriptions
0,30843639,Christopher Isherwood,"[take, see, take, find, try, meet, suggest, pr...","[secure, leave, publish, remain, see, entreat,...","[death, life, family]",artist,NaN,NaN,NaN,NaN,False,"[take, see, take, find, try, meet, suggest, pr..."
1,30843639,Caspar,[disappear],NaN,"[one, rentboys]",NaN,NaN,NaN,NaN,NaN,False,"[disappear, one, rentboys]"
2,30843639,Wilfrid Landauer,[burn],NaN,"[see, last]",owner,NaN,NaN,NaN,NaN,False,"[burn, see, last]"
3,30843639,Gerhardt,[advise],NaN,[brother],NaN,NaN,NaN,NaN,NaN,False,"[advise, brother]"
4,30843639,Heinz Neddermayer,"[decide, express]","[deny, leave, advise, welcome, sentence, arrest]",[brother],street sweeper,NaN,NaN,NaN,NaN,False,"[decide, express, deny, leave, advise, welcome..."
...,...,...,...,...,...,...,...,...,...,...,...,...
162332,22430724,Colleen Peck,"[tell, express, appear]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,"[tell, express, appear]"
162333,22430724,Mike Pomeroy,"[prof, show, manage, tell, break, end, end, co...","[escape, tell, go, choose]","[reluctant, rans]",journalist,NaN,NaN,NaN,NaN,True,"[prof, show, manage, tell, break, end, end, co..."
162334,22430724,Adam Bennett,"[tease, realize]",NaN,NaN,producer,NaN,NaN,NaN,NaN,True,"[tease, realize]"
162335,22430724,Jerry Barnes,"[hire, remain]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,"[hire, remain]"


In [20]:
# For each movie_id in descriptions, synchronize the names and store in name_sync_df
name_sync_df = pd.DataFrame(columns=['movie_id', 'name_sync'])
for movie_id in descriptions['movie_id'].unique():
    name_sync_df = pd.concat([name_sync_df, pd.DataFrame({
        'movie_id': [movie_id], 
        'name_sync': [synchronize_name(movie_id, char_df, descriptions, col_name='character')]})], 
        ignore_index=True)

# Index name_sync by movie_id
name_sync_df = name_sync_df.set_index('movie_id')


KeyboardInterrupt: 

In [ ]:
# For each character in descriptions, get the corresponding name_sync
descriptions['plot_name'] = descriptions['character']
descriptions['character'] = descriptions[['movie_id', 'character']].apply(
    lambda x: name_sync_df.loc[x['movie_id']].values[0][x['character']], axis=1)


4. Merge with character metadata. 

In [ ]:
# For each movie_id in descriptions, get the corresponding Freebase ID (except if you can't find it)
descriptions['freebase_id'] = descriptions['movie_id'].apply(
    lambda x: movie_df.loc[x]['Freebase ID'] if x in movie_df.index else np.nan)

# Merge descriptions with char_df on character name and movie_id
descriptions = descriptions.merge(char_df, left_on=['character', 'movie_id'], right_on=[
                                  'Character name', 'Wikipedia ID'], how='left')

# Drop columns movie_id, character
descriptions = descriptions.drop(
    ['movie_id', 'character', 'freebase_id'], axis=1)

# Reorder columns
cols = descriptions.columns.tolist()
cols = [col for col in cols if col not in ['Wikipedia ID', 'Character name']]
cols = ['Wikipedia ID', 'Character name'] + cols
descriptions = descriptions[cols]

# Save descriptions to csv
descriptions.to_csv('Data/CoreNLP/full_descriptions.csv',
                    sep='\t', index=False)


5. Aggregate all descriptions over all movies for each character.

In [ ]:
# Make a new DataFrame with a single character per row
char_descriptions = pd.DataFrame(
    descriptions[['Character name', 'Freebase character ID']].drop_duplicates())

# Aggregate all agent_verbs together
agent_verbs = descriptions.groupby(['Freebase character ID'])['agent_verbs'].aggregate(
    lambda x: list(itertools.chain.from_iterable(x.dropna())))
char_descriptions = char_descriptions.merge(
    agent_verbs, left_on='Freebase character ID', right_index=True, how='left')

# Aggregate all patient_verbs together
patient_verbs = descriptions.groupby(['Freebase character ID'])['patient_verbs'].aggregate(
    lambda x: list(itertools.chain.from_iterable(x.dropna())))
char_descriptions = char_descriptions.merge(
    patient_verbs, left_on='Freebase character ID', right_index=True, how='left')

# Aggregate all attributes together
attributes = descriptions.groupby(['Freebase character ID'])['attributes'].aggregate(
    lambda x: list(itertools.chain.from_iterable(x.dropna())))
char_descriptions = char_descriptions.merge(
    attributes, left_on='Freebase character ID', right_index=True, how='left')

# Aggregate all titles together into a list of titles
titles = descriptions.groupby(['Freebase character ID'])['title'].aggregate(
    lambda x: list(x.dropna()))
char_descriptions = char_descriptions.merge(
    titles, left_on='Freebase character ID', right_index=True, how='left')

# Concatenate all agent_verbs, patient_verbs, attributes, titles into a single list of descriptions
char_descriptions['descriptions'] = char_descriptions[['agent_verbs', 'patient_verbs',
                                                       'attributes', 'title']].apply(lambda x: list(itertools.chain.from_iterable(x.dropna())), axis=1)

# Replace all empty lists with NaN
char_descriptions = char_descriptions.dropna(subset=['Character name', 'Freebase character ID']) 
char_descriptions['agent_verbs'] = char_descriptions['agent_verbs'].apply(lambda x: np.nan if (type(x) == list and len(x) == 0) else x).copy()
char_descriptions['patient_verbs'] = char_descriptions['patient_verbs'].apply(lambda x: np.nan if (type(x) == list and len(x) == 0) else x).copy()
char_descriptions['attributes'] = char_descriptions['attributes'].apply(lambda x: np.nan if (type(x) == list and len(x) == 0) else x).copy()
char_descriptions['title'] = char_descriptions['title'].apply(lambda x: np.nan if (type(x) == list and len(x) == 0) else x).copy()
char_descriptions['descriptions'] = char_descriptions['descriptions'].apply(lambda x: np.nan if (type(x) == list and len(x) == 0) else x).copy()


# Save char_descriptions to csv
char_descriptions.to_csv(
    'Data/CoreNLP/char_descriptions.csv', sep='\t', index=False)


#### 4.2. Processing relations

We synchronize the names of characters in relationships to their names in the character metadata. 

In [ ]:
# For each movie_id in relations, synchronize the subject and object names and store in name_rel_df
name_rel_df = pd.DataFrame(columns=['movie_id', 'subject_sync', 'object_sync'])
for movie_id in relations['movie_id'].unique():
    name_rel_df = pd.concat([
        name_rel_df, 
        pd.DataFrame({
            'movie_id': [movie_id], 
            'subject_sync': [synchronize_name(movie_id, char_df, df=relations, col_name='subject')], 
            'object_sync': [synchronize_name(movie_id, char_df, df=relations, col_name='object')]})], ignore_index=True)


# Index name_sync by movie_id
name_rel_df = name_rel_df.set_index('movie_id')


In [ ]:
# For each character in relations, get the corresponding name_sync (except if you can't find the character in the given movie)
relations['subject'] = relations[['movie_id', 'subject']].apply(
    lambda x: name_rel_df.loc[x['movie_id']]['subject_sync'][x['subject']] if x['subject'] in name_rel_df.loc[x['movie_id']]['subject_sync'] else np.nan, axis=1)
relations['object'] = relations[['movie_id', 'object']].apply(
    lambda x: name_rel_df.loc[x['movie_id']]['object_sync'][x['object']] if x['object'] in name_rel_df.loc[x['movie_id']]['object_sync']else np.nan, axis=1)

# Rename columns to Wikipedia ID, Subject, Object, Romance
relations = relations.rename(columns={'movie_id': 'Wikipedia ID', 'subject': 'Subject', 'object': 'Object', 'romance': 'Romance'})

# Add subject and object Freebase character IDs and rename them to Subject/object freebase character ID column
relations = relations.merge(
    char_df[['Wikipedia ID', 'Character name', 'Freebase character ID']], 
    left_on=['Subject', 'Wikipedia ID'], 
    right_on=['Character name', 'Wikipedia ID'], 
    how='left').rename(columns={'Freebase character ID': 'Subject freebase character ID'})
relations = relations.merge(
    char_df[['Wikipedia ID', 'Character name', 'Freebase character ID']], 
    left_on=['Object', 'Wikipedia ID'], 
    right_on=['Character name', 'Wikipedia ID'], 
    how='left').rename(columns={'Freebase character ID': 'Object freebase character ID'})

# Drop columns character name_x and character name_y
relations = relations.drop(columns=['Character name_x', 'Character name_y']).copy()

# Save relations to csv
relations.to_csv('Data/CoreNLP/char_relations.csv', sep='\t', index=False)